In [1]:
!pip install xgboost
!pip install geopandas
!pip install folium
!pip install lightgbm
!pip install utm
!pip install geojson 
!pip install haversine

     |████████████████████████████████| 972kB 3.3MB/s 
     |████████████████████████████████| 10.9MB 18.0MB/s 
     |████████████████████████████████| 14.8MB 302kB/s 
  Created wheel for utm: filename=utm-0.6.0-cp36-none-any.whl size=6095 sha256=d1a49489b7023a71e9866593751301714ef1a9fd73d3dff199ce3d025b1f1255
  Stored in directory: /root/.cache/pip/wheels/9a/76/b2/58ad7ade8b0475c577f80ee8023d7164fd561bb45c20b4beab
Successfully built utm


In [ ]:
# 시각화 도구 설치
!pip install plotnine #R 시각화 
!pip install folium #지도 시각화 

# 그래프에서 한글표현을 위해 폰트를 설치합니다.
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [ ]:
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()

NanumGothic Eco


In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from haversine import haversine
from shapely.geometry import Point, Polygon, LineString
import math
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
Bike_Station=pd.read_csv('/content/drive/My Drive/Compas/compas/SBJ_2007_001/02.자전거스테이션.csv')
Bike_Station

,Station_ID,STATION_NAME,거치대 수량,위도,경도
0,101,어울림마을 701동 앞,20,37.654775,126.834584
1,103,대림e-편한세상106동,20,37.660442,126.840377
2,104,탄현마을8단지,25,37.698523,126.766042
3,105,KT 덕양지사 앞,20,37.655244,126.839261
4,106,원당역 앞 공영주차장,30,37.653410,126.842530
...,...,...,...,...,...
159,350,★하이파크5단지 502동앞 버스정류장,20,37.697867,126.753089
160,351,◆일산에듀포레 푸르지오,40,37.702259,126.767231
161,352,◆꿈에그린203동앞,40,37.666425,126.749244
162,353,◆꿈에그린106동앞,40,37.666720,126.750784


In [ ]:
Bike_Station['Station_ID'].unique(), len(Bike_Station['Station_ID'].unique())

(array([101, 103, 104, 105, 106, 110, 111, 112, 113, 114, 115, 116, 118,
        119, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 137,
        138, 139, 140, 141, 142, 143, 144, 146, 148, 151, 152, 153, 154,
        155, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
        173, 176, 177, 178, 201, 202, 203, 204, 205, 206, 207, 210, 211,
        212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224,
        225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
        238, 239, 240, 241, 242, 245, 246, 247, 248, 249, 250, 251, 252,
        253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265,
        301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313,
        314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 326, 327,
        328, 329, 330, 331, 333, 334, 339, 340, 341, 342, 343, 345, 346,
        347, 348, 349, 350, 351, 352, 353, 992]), 164)

In [ ]:
Bike_Station[Bike_Station['Station_ID']==155]

,Station_ID,STATION_NAME,거치대 수량,위도,경도
39,155,◆향동 덕은교 155,40,37.591856,126.882963


In [ ]:
bike_rental_month = pd.read_csv('/content/drive/My Drive/Compas/compas/준수씨 자전거 자료/Koyang_Bike(정리중)_CSV및 다양한 파일/월별 대여 스테이션 이용 현황(완성).csv')
bike_rental_month

,Unnamed: 0,대여 시간(M),대여 스테이션 번호,이용 횟수(월)
0,0,2017-01,101,80
1,1,2017-01,103,33
2,2,2017-01,104,152
3,3,2017-01,105,27
4,4,2017-01,106,62
...,...,...,...,...
5611,5611,2019-12,349,133
5612,5612,2019-12,350,54
5613,5613,2019-12,351,157
5614,5614,2019-12,992,19


In [ ]:
bike_rental_month['대여 스테이션 번호'].unique(), len(bike_rental_month['대여 스테이션 번호'].unique())

(array([101, 103, 104, 105, 106, 110, 111, 112, 113, 114, 115, 116, 118,
        119, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 137,
        138, 139, 140, 141, 142, 143, 144, 146, 148, 161, 162, 163, 164,
        165, 166, 167, 168, 169, 170, 171, 172, 173, 176, 177, 178, 201,
        202, 203, 204, 205, 206, 207, 210, 211, 212, 213, 214, 215, 216,
        217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229,
        230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 245, 246,
        247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259,
        260, 261, 262, 263, 264, 265, 301, 302, 303, 304, 305, 306, 307,
        308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320,
        321, 322, 323, 324, 326, 327, 328, 329, 330, 331, 333, 334, 339,
        340, 341, 342, 343, 345, 346, 347, 348, 349, 350, 351, 992, 998]), 156)

In [ ]:
bike_return_month = pd.read_csv('/content/drive/My Drive/Compas/compas/준수씨 자전거 자료/Koyang_Bike(정리중)_CSV및 다양한 파일/월별 반납 스테이션 이용 현황(완성).csv')
bike_return_month

,Unnamed: 0,Unnamed: 0.1,반납 시간(M),반납 스테이션 번호,이용 횟수(월)
0,0,0,2017-01,0,3826
1,1,1,2017-01,101,109
2,2,2,2017-01,103,44
3,3,3,2017-01,104,89
4,4,4,2017-01,105,17
...,...,...,...,...,...
5683,5683,153,2019-12,350,37
5684,5684,154,2019-12,351,149
5685,5685,155,2019-12,992,19
5686,5686,156,2019-12,998,1


In [ ]:
bike_return_month['반납 스테이션 번호'].unique()

array([  0, 101, 103, 104, 105, 106, 110, 111, 112, 113, 114, 115, 116,
       118, 119, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133,
       137, 138, 139, 140, 141, 142, 143, 144, 146, 148, 161, 162, 163,
       164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 176, 177, 178,
       201, 202, 203, 204, 205, 206, 207, 210, 211, 212, 213, 214, 215,
       216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228,
       229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 245,
       246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258,
       259, 260, 261, 262, 263, 264, 265, 301, 302, 303, 304, 305, 306,
       307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319,
       320, 321, 322, 323, 324, 326, 327, 328, 329, 330, 331, 333, 334,
       339, 340, 341, 342, 343, 345, 346, 347, 348, 349, 350, 351, 992,
       998, 999])

In [ ]:
bike_rental_month_groupby = dict(list(bike_rental_month.groupby('대여 스테이션 번호')))
bike_rental_month_groupby

{101:       Unnamed: 0 대여 시간(M)  대여 스테이션 번호  이용 횟수(월)
 0              0  2017-01         101        80
 156          156  2017-02         101        74
 312          312  2017-03         101       201
 468          468  2017-04         101       306
 624          624  2017-05         101       437
 780          780  2017-06         101       435
 936          936  2017-07         101       229
 1092        1092  2017-08         101       289
 1248        1248  2017-09         101       415
 1404        1404  2017-10         101       312
 1560        1560  2017-11         101       160
 1716        1716  2017-12         101        75
 1872        1872  2018-01         101        53
 2028        2028  2018-02         101        48
 2184        2184  2018-03         101       136
 2340        2340  2018-04         101       203
 2496        2496  2018-05         101       274
 2652        2652  2018-06         101       333
 2808        2808  2018-07         101       295
 2964        29

In [ ]:
len(bike_rental_month_groupby.keys())

156

In [ ]:
bike_rental_month_groupby[101]

,Unnamed: 0,대여 시간(M),대여 스테이션 번호,이용 횟수(월)
0,0,2017-01,101,80
156,156,2017-02,101,74
312,312,2017-03,101,201
468,468,2017-04,101,306
624,624,2017-05,101,437
780,780,2017-06,101,435
936,936,2017-07,101,229
1092,1092,2017-08,101,289
1248,1248,2017-09,101,415
1404,1404,2017-10,101,312


In [ ]:
bike_rental_month_groupby[351]

,Unnamed: 0,대여 시간(M),대여 스테이션 번호,이용 횟수(월)
153,153,2017-01,351,0
309,309,2017-02,351,0
465,465,2017-03,351,0
621,621,2017-04,351,0
777,777,2017-05,351,0
933,933,2017-06,351,0
1089,1089,2017-07,351,0
1245,1245,2017-08,351,0
1401,1401,2017-09,351,0
1557,1557,2017-10,351,0


In [ ]:
sum([1 if x != 0 else 0 for x in bike_rental_month_groupby[351]['이용 횟수(월)']])

4

In [ ]:
sum(bike_rental_month_groupby[351]['이용 횟수(월)'])

473

In [ ]:
sum(bike_rental_month_groupby[351]['이용 횟수(월)']) / sum([1 if x != 0 else 0 for x in bike_rental_month_groupby[351]['이용 횟수(월)']])

118.25

In [ ]:
bike_rental_month_groupby[351]['이용 횟수(월)'].mean()

13.13888888888889

In [ ]:
from collections import defaultdict

rental_month = defaultdict(lambda x: np.nan)

for k in bike_rental_month_groupby.keys():
  denom = sum([1 if x != 0 else 0 for x in bike_rental_month_groupby[k]['이용 횟수(월)']])
  num = sum(bike_rental_month_groupby[k]['이용 횟수(월)'])
  rental_month[k] = (num/denom)
rental_month

defaultdict(<function __main__.<lambda>>,
            {101: 232.38888888888889,
             103: 125.97222222222223,
             104: 372.27777777777777,
             105: 77.94444444444444,
             106: 123.44444444444444,
             110: 392.72222222222223,
             111: 263.30555555555554,
             112: 225.61111111111111,
             113: 333.1111111111111,
             114: 372.80555555555554,
             115: 245.72222222222223,
             116: 326.8611111111111,
             118: 491.6388888888889,
             119: 402.19444444444446,
             121: 324.4166666666667,
             123: 179.41666666666666,
             124: 222.38888888888889,
             125: 252.42857142857142,
             126: 187.75,
             127: 314.6111111111111,
             128: 172.86111111111111,
             129: 268.19444444444446,
             130: 265.9166666666667,
             131: 258.69444444444446,
             133: 138.27777777777777,
             137: 264.02777

In [ ]:
rental_month = pd.DataFrame(rental_month.items(), columns = ['스테이션', '대여_월평균'])
rental_month

,스테이션,대여_월평균
0,101,232.388889
1,103,125.972222
2,104,372.277778
3,105,77.944444
4,106,123.444444
...,...,...
151,349,209.222222
152,350,116.388889
153,351,118.250000
154,992,26.444444


## 반납

In [ ]:
bike_return_month_groupby = dict(list(bike_return_month.groupby('반납 스테이션 번호')))
bike_return_month_groupby

{0:       Unnamed: 0  Unnamed: 0.1 반납 시간(M)  반납 스테이션 번호  이용 횟수(월)
 0              0             0  2017-01           0      3826
 158          158             0  2017-02           0      3428
 316          316             0  2017-03           0      6385
 474          474             0  2017-04           0      9795
 632          632             0  2017-05           0     12289
 790          790             0  2017-06           0     13115
 948          948             0  2017-07           0     10690
 1106        1106             0  2017-08           0     11113
 1264        1264             0  2017-09           0     13439
 1422        1422             0  2017-10           0     11915
 1580        1580             0  2017-11           0      7820
 1738        1738             0  2017-12           0      3995
 1896        1896             0  2018-01           0      3394
 2054        2054             0  2018-02           0      3137
 2212        2212             0  2018-03           0

In [ ]:
from collections import defaultdict

return_month = defaultdict(lambda x: np.nan)

for k in bike_return_month_groupby.keys():
  denom = sum([1 if x != 0 else 0 for x in bike_return_month_groupby[k]['이용 횟수(월)']])
  num = sum(bike_return_month_groupby[k]['이용 횟수(월)'])
  return_month[k] = (num/denom)
return_month

defaultdict(<function __main__.<lambda>>,
            {0: 11062.055555555555,
             101: 255.75,
             103: 142.25,
             104: 259.1666666666667,
             105: 74.88888888888889,
             106: 65.47222222222223,
             110: 392.80555555555554,
             111: 245.41666666666666,
             112: 184.47222222222223,
             113: 281.3888888888889,
             114: 314.6111111111111,
             115: 189.69444444444446,
             116: 295.3888888888889,
             118: 430.05555555555554,
             119: 360.0,
             121: 305.44444444444446,
             123: 171.19444444444446,
             124: 197.11111111111111,
             125: 208.97142857142856,
             126: 157.36111111111111,
             127: 189.83333333333334,
             128: 105.91666666666667,
             129: 246.0,
             130: 172.25,
             131: 171.5,
             133: 165.75,
             137: 287.3333333333333,
             138: 181.5,
   

In [ ]:
return_month = pd.DataFrame(return_month.items(), columns = ['스테이션', '반납_월평균'])
return_month

,스테이션,반납_월평균
0,0,11062.055556
1,101,255.750000
2,103,142.250000
3,104,259.166667
4,105,74.888889
...,...,...
153,350,101.722222
154,351,101.500000
155,992,22.083333
156,998,10.500000


## Merge

In [ ]:
for x in bike_rental_month_groupby.keys():
  if x not in bike_return_month_groupby.keys():
    print(x)

In [ ]:
for x in bike_return_month_groupby.keys():
  if x not in bike_rental_month_groupby.keys():
    print(x)

0
999


In [ ]:
# 0과 999 제외 
rental_month.merge(return_month, on = '스테이션')

,스테이션,대여_월평균,반납_월평균
0,101,232.388889,255.750000
1,103,125.972222,142.250000
2,104,372.277778,259.166667
3,105,77.944444,74.888889
4,106,123.444444,65.472222
...,...,...,...
151,349,209.222222,176.000000
152,350,116.388889,101.722222
153,351,118.250000,101.500000
154,992,26.444444,22.083333


In [ ]:
# 0와 999 포함 
station_merge = rental_month.merge(return_month, how = 'outer', on = '스테이션')
station_merge

,스테이션,대여_월평균,반납_월평균
0,101,232.388889,255.750000
1,103,125.972222,142.250000
2,104,372.277778,259.166667
3,105,77.944444,74.888889
4,106,123.444444,65.472222
...,...,...,...
153,351,118.250000,101.500000
154,992,26.444444,22.083333
155,998,12.500000,10.500000
156,0,NaN,11062.055556


In [ ]:
station_merge['총 이용_월평균'] = station_merge['대여_월평균'].fillna(0) + station_merge['반납_월평균'] 
station_merge

,스테이션,대여_월평균,반납_월평균,총 이용_월평균
0,101,232.388889,255.750000,488.138889
1,103,125.972222,142.250000,268.222222
2,104,372.277778,259.166667,631.444444
3,105,77.944444,74.888889,152.833333
4,106,123.444444,65.472222,188.916667
...,...,...,...,...
153,351,118.250000,101.500000,219.750000
154,992,26.444444,22.083333,48.527778
155,998,12.500000,10.500000,23.000000
156,0,NaN,11062.055556,11062.055556


In [ ]:
station_merge.to_csv('/content/drive/My Drive/Compas/compas/csv 파일 여기로!/스테이션 이용_월평균.csv', index = False)

## 기존 LSCP 수정

In [ ]:
#평균 ver bike point 
lscp_prev = pd.read_pickle('/content/drive/My Drive/Compas/compas/csv 파일 여기로!/bikepoint_lscp_all_300_도로명추가.pickle')
lscp_prev

,좌표,대여수,반납수,총 이용횟수,스테이션,주거인구,버스_승하차수,지하철_승하차수,위도,경도,DN_avg,Road_yes,Space_count,Culture_count,Sport_count,KINTEX,Hosu_yes,Park_count,CD_01003_다가구주택,CD_02003_다세대주택,CD_03999_기타제1종근생,CD_03005_의원,CD_01001_단독주택,CD_13100_일반공장,CD_16006_정비공장,CD_03001_소매점,CD_17007_온실,CD_16010_주기장,CD_04001_일반음식점,CD_03104_소방서,CD_14999_기타창고시설,CD_04005_제조업소,CD_04010_학원,CD_04004_서점,CD_14001_창고,CD_04999_기타제2종근생,CD_19003_군사시설,CD_04006_수리점,CD_02002_연립주택,CD_01002_다중주택,...,CD_03014_치과의원,CD_12999_기타위락시설,CD_05301_예식장,CD_13200_공해공장,CD_19006_전신전화국,CD_04107_골프연습장,CD_07101_종합병원,CD_06204_대형점,CD_04022_멀티미디어문화콘텐츠 설비제공업소,CD_07102_산부인과병원,CD_06303_여객자동차터미널,CD_15006_액화가스판매소,CD_08601_청소년수련원(관),CD_15999_기타위험물저장처리시설,CD_04405_출판사,CD_16007_운전학원,CD_03019_탁구장,CD_03017_접골원,CD_03018_조산원,CD_04023_복합유통제공업소,CD_11299_기타관광숙박시설,CD_04008_삭진관,CD_09005_테니스장,CD_03106_전신전화국,CD_05303_공회당,CD_05699_기타동.식물원,CD_04399_기타공연장,CD_05403_체육관,CD_15007_유독물보관저장시설,CD_05505_산업전시장,CD_04021_세탁소,CD_09011_골프연습장,CD_20002_납골당,CD_06201_시장,CD_17105_부화장,CD_08699_기타생활권수련시설,CD_03109_공공도서관,CD_27999,CD_90002,CD_05502_미술관
0,"(37.59052639593674, 126.88041634904563)",0.0,0.0,0.0,[155],74.0,0.0,0.0,37.590526,126.880416,16,1,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0,0,1,0,9,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"(37.590546726423185, 126.88154829019997)",0.0,0.0,0.0,[155],36.0,0.0,0.0,37.590547,126.881548,17,1,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,1,0,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"(37.59142713034578, 126.88039079257342)",0.0,0.0,0.0,[155],135.0,0.0,0.0,37.591427,126.880391,18,1,0,0,0,0,0,0,0,0,0,0,19,0,0,1,0,0,2,0,10,0,0,0,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"(37.59144746148929, 126.88152274735107)",0.0,0.0,0.0,[155],82.0,0.0,0.0,37.591447,126.881523,21,1,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,1,0,6,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"(37.59146778178655, 126.88265470326604)",0.0,0.0,0.0,[155],36.0,0.0,0.0,37.591468,126.882655,21,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,"(37.70445091519225, 126.75467703178417)",6093.0,4582.0,10675.0,[346],3562.0,5665.0,0.0,37.704451,126.754677,28,0,0,0,3,0,0,1,0,12,1,0,8,0,0,2,0,0,0,0,2,0,0,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3230,"(37.704472506609626, 126.75581060819373)",6093.0,4582.0,10675.0,[346],2661.0,3486.0,0.0,37.704473,126.755811,27,0,0,0,0,0,0,1,0,12,1,0,9,0,0,9,0,0,0,0,3,0,0,0,2,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3231,"(37.70466633943072, 126.7660128502553)",473.0,406.0,879.0,[351],7713.0,21185.0,0.0,37.704666,126.766013,31,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,6,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3232,"(37.70468782197291, 126.76714643873707)",473.0,406.0,879.0,[351],6526.0,21185.0,0.0,37.704688,126.767146,33,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,4,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
lscp_base = lscp_prev[['좌표', '스테이션']]
lscp_base

,좌표,스테이션
0,"(37.59052639593674, 126.88041634904563)",[155]
1,"(37.590546726423185, 126.88154829019997)",[155]
2,"(37.59142713034578, 126.88039079257342)",[155]
3,"(37.59144746148929, 126.88152274735107)",[155]
4,"(37.59146778178655, 126.88265470326604)",[155]
...,...,...
3229,"(37.70445091519225, 126.75467703178417)",[346]
3230,"(37.704472506609626, 126.75581060819373)",[346]
3231,"(37.70466633943072, 126.7660128502553)",[351]
3232,"(37.70468782197291, 126.76714643873707)",[351]


In [ ]:
lscp_prev = lscp_prev.drop(['대여수', '반납수', '총 이용횟수'], axis = 1)
lscp_prev

,좌표,스테이션,주거인구,버스_승하차수,지하철_승하차수,위도,경도,DN_avg,Road_yes,Space_count,Culture_count,Sport_count,KINTEX,Hosu_yes,Park_count,CD_01003_다가구주택,CD_02003_다세대주택,CD_03999_기타제1종근생,CD_03005_의원,CD_01001_단독주택,CD_13100_일반공장,CD_16006_정비공장,CD_03001_소매점,CD_17007_온실,CD_16010_주기장,CD_04001_일반음식점,CD_03104_소방서,CD_14999_기타창고시설,CD_04005_제조업소,CD_04010_학원,CD_04004_서점,CD_14001_창고,CD_04999_기타제2종근생,CD_19003_군사시설,CD_04006_수리점,CD_02002_연립주택,CD_01002_다중주택,CD_04402_사무소,CD_04403_부동산중개업소,CD_08102_중학교,...,CD_03014_치과의원,CD_12999_기타위락시설,CD_05301_예식장,CD_13200_공해공장,CD_19006_전신전화국,CD_04107_골프연습장,CD_07101_종합병원,CD_06204_대형점,CD_04022_멀티미디어문화콘텐츠 설비제공업소,CD_07102_산부인과병원,CD_06303_여객자동차터미널,CD_15006_액화가스판매소,CD_08601_청소년수련원(관),CD_15999_기타위험물저장처리시설,CD_04405_출판사,CD_16007_운전학원,CD_03019_탁구장,CD_03017_접골원,CD_03018_조산원,CD_04023_복합유통제공업소,CD_11299_기타관광숙박시설,CD_04008_삭진관,CD_09005_테니스장,CD_03106_전신전화국,CD_05303_공회당,CD_05699_기타동.식물원,CD_04399_기타공연장,CD_05403_체육관,CD_15007_유독물보관저장시설,CD_05505_산업전시장,CD_04021_세탁소,CD_09011_골프연습장,CD_20002_납골당,CD_06201_시장,CD_17105_부화장,CD_08699_기타생활권수련시설,CD_03109_공공도서관,CD_27999,CD_90002,CD_05502_미술관
0,"(37.59052639593674, 126.88041634904563)",[155],74.0,0.0,0.0,37.590526,126.880416,16,1,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0,0,1,0,9,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"(37.590546726423185, 126.88154829019997)",[155],36.0,0.0,0.0,37.590547,126.881548,17,1,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,1,0,4,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"(37.59142713034578, 126.88039079257342)",[155],135.0,0.0,0.0,37.591427,126.880391,18,1,0,0,0,0,0,0,0,0,0,0,19,0,0,1,0,0,2,0,10,0,0,0,0,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"(37.59144746148929, 126.88152274735107)",[155],82.0,0.0,0.0,37.591447,126.881523,21,1,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,1,0,6,0,0,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"(37.59146778178655, 126.88265470326604)",[155],36.0,0.0,0.0,37.591468,126.882655,21,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,"(37.70445091519225, 126.75467703178417)",[346],3562.0,5665.0,0.0,37.704451,126.754677,28,0,0,0,3,0,0,1,0,12,1,0,8,0,0,2,0,0,0,0,2,0,0,0,1,2,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3230,"(37.704472506609626, 126.75581060819373)",[346],2661.0,3486.0,0.0,37.704473,126.755811,27,0,0,0,0,0,0,1,0,12,1,0,9,0,0,9,0,0,0,0,3,0,0,0,2,3,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3231,"(37.70466633943072, 126.7660128502553)",[351],7713.0,21185.0,0.0,37.704666,126.766013,31,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,6,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3232,"(37.70468782197291, 126.76714643873707)",[351],6526.0,21185.0,0.0,37.704688,126.767146,33,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,4,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
station_merge

,스테이션,대여_월평균,반납_월평균,총 이용_월평균
0,101,232.388889,255.750000,488.138889
1,103,125.972222,142.250000,268.222222
2,104,372.277778,259.166667,631.444444
3,105,77.944444,74.888889,152.833333
4,106,123.444444,65.472222,188.916667
...,...,...,...,...
153,351,118.250000,101.500000,219.750000
154,992,26.444444,22.083333,48.527778
155,998,12.500000,10.500000,23.000000
156,0,NaN,11062.055556,11062.055556


In [ ]:
station_merge[station_merge['스테이션'] == 155]

,스테이션,대여_월평균,반납_월평균,총 이용_월평균


### 1. 평균

In [ ]:
lscp_base['대여_월평균'] = np.zeros(len(lscp_prev))
lscp_base['반납_월평균'] = np.zeros(len(lscp_prev))
lscp_base['총 이용_월평균'] = np.zeros(len(lscp_prev))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
# 평균 ver
for lscp_idx, x in enumerate(lscp_base['스테이션']):
  rent = []
  return_ = []
  total = []
  for st_idx, st in enumerate(station_merge['스테이션']):
    if st in x: 
      rent.append(station_merge['대여_월평균'].iloc[st_idx])
      return_.append(station_merge['반납_월평균'].iloc[st_idx])
      total.append(station_merge['총 이용_월평균'].iloc[st_idx])
  if len(rent) != 0:
    lscp_base['대여_월평균'].iloc[lscp_idx] = sum(rent) / len(rent)
  else:
    lscp_base['대여_월평균'].iloc[lscp_idx] = sum(rent)
  if len(return_) != 0:
    lscp_base['반납_월평균'].iloc[lscp_idx] = sum(return_) / len(return_)
  else:
    lscp_base['반납_월평균'].iloc[lscp_idx] = sum(return_)
  if len(total) != 0:
    lscp_base['총 이용_월평균'].iloc[lscp_idx] = sum(total) / len(total)
  else:
    lscp_base['총 이용_월평균'].iloc[lscp_idx] = sum(total)
lscp_base

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:

,좌표,스테이션,대여_월평균,반납_월평균,총 이용_월평균
0,"(37.59052639593674, 126.88041634904563)",[155],0.00,0.000000,0.000000
1,"(37.590546726423185, 126.88154829019997)",[155],0.00,0.000000,0.000000
2,"(37.59142713034578, 126.88039079257342)",[155],0.00,0.000000,0.000000
3,"(37.59144746148929, 126.88152274735107)",[155],0.00,0.000000,0.000000
4,"(37.59146778178655, 126.88265470326604)",[155],0.00,0.000000,0.000000
...,...,...,...,...,...
3229,"(37.70445091519225, 126.75467703178417)",[346],169.25,127.277778,296.527778
3230,"(37.704472506609626, 126.75581060819373)",[346],169.25,127.277778,296.527778
3231,"(37.70466633943072, 126.7660128502553)",[351],118.25,101.500000,219.750000
3232,"(37.70468782197291, 126.76714643873707)",[351],118.25,101.500000,219.750000


In [ ]:
# # 합 ver
# for lscp_idx, x in enumerate(lscp_base['스테이션']):
#   rent = []
#   return_ = []
#   total = []
#   for st_idx, st in enumerate(station_merge['스테이션']):
#     if st in x: 
#       rent.append(station_merge['대여_월평균'].iloc[st_idx])
#       return_.append(station_merge['반납_월평균'].iloc[st_idx])
#       total.append(station_merge['총 이용_월평균'].iloc[st_idx])
#   if len(rent) != 0:
#     lscp_base['대여_월평균'].iloc[lscp_idx] = sum(rent)
#   else:
#     lscp_base['대여_월평균'].iloc[lscp_idx] = sum(rent)
#   if len(return_) != 0:
#     lscp_base['반납_월평균'].iloc[lscp_idx] = sum(return_)
#   else:
#     lscp_base['반납_월평균'].iloc[lscp_idx] = sum(return_)
#   if len(total) != 0:
#     lscp_base['총 이용_월평균'].iloc[lscp_idx] = sum(total)
#   else:
#     lscp_base['총 이용_월평균'].iloc[lscp_idx] = sum(total)
# lscp_base

In [ ]:
lscp_base[350:351]

,좌표,스테이션,대여_월평균,반납_월평균,총 이용_월평균
350,"(37.62221275756768, 126.83872881153121)","[128, 129]",220.527778,175.958333,396.486111


In [ ]:
station_merge[station_merge['스테이션'] == 128], station_merge[station_merge['스테이션'] == 129]

(    스테이션      대여_월평균      반납_월평균    총 이용_월평균
 20   128  172.861111  105.916667  278.777778,
     스테이션      대여_월평균  반납_월평균    총 이용_월평균
 21   129  268.194444   246.0  514.194444)

In [ ]:
(278.777778 +  514.194444)

792.972222

In [ ]:
(278.777778 +  514.194444) / 2

396.486111

In [ ]:
lscp_base = lscp_base.drop('스테이션', axis = 1)

In [ ]:
lscp_new = lscp_base.merge(lscp_prev, on = '좌표')
lscp_new

,좌표,대여_월평균,반납_월평균,총 이용_월평균,스테이션,주거인구,버스_승하차수,지하철_승하차수,위도,경도,DN_avg,Road_yes,Space_count,Culture_count,Sport_count,KINTEX,Hosu_yes,Park_count,CD_01003_다가구주택,CD_02003_다세대주택,CD_03999_기타제1종근생,CD_03005_의원,CD_01001_단독주택,CD_13100_일반공장,CD_16006_정비공장,CD_03001_소매점,CD_17007_온실,CD_16010_주기장,CD_04001_일반음식점,CD_03104_소방서,CD_14999_기타창고시설,CD_04005_제조업소,CD_04010_학원,CD_04004_서점,CD_14001_창고,CD_04999_기타제2종근생,CD_19003_군사시설,CD_04006_수리점,CD_02002_연립주택,CD_01002_다중주택,...,CD_03014_치과의원,CD_12999_기타위락시설,CD_05301_예식장,CD_13200_공해공장,CD_19006_전신전화국,CD_04107_골프연습장,CD_07101_종합병원,CD_06204_대형점,CD_04022_멀티미디어문화콘텐츠 설비제공업소,CD_07102_산부인과병원,CD_06303_여객자동차터미널,CD_15006_액화가스판매소,CD_08601_청소년수련원(관),CD_15999_기타위험물저장처리시설,CD_04405_출판사,CD_16007_운전학원,CD_03019_탁구장,CD_03017_접골원,CD_03018_조산원,CD_04023_복합유통제공업소,CD_11299_기타관광숙박시설,CD_04008_삭진관,CD_09005_테니스장,CD_03106_전신전화국,CD_05303_공회당,CD_05699_기타동.식물원,CD_04399_기타공연장,CD_05403_체육관,CD_15007_유독물보관저장시설,CD_05505_산업전시장,CD_04021_세탁소,CD_09011_골프연습장,CD_20002_납골당,CD_06201_시장,CD_17105_부화장,CD_08699_기타생활권수련시설,CD_03109_공공도서관,CD_27999,CD_90002,CD_05502_미술관
0,"(37.59052639593674, 126.88041634904563)",0.00,0.000000,0.000000,[155],74.0,0.0,0.0,37.590526,126.880416,16,1,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0,0,1,0,9,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"(37.590546726423185, 126.88154829019997)",0.00,0.000000,0.000000,[155],36.0,0.0,0.0,37.590547,126.881548,17,1,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,1,0,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"(37.59142713034578, 126.88039079257342)",0.00,0.000000,0.000000,[155],135.0,0.0,0.0,37.591427,126.880391,18,1,0,0,0,0,0,0,0,0,0,0,19,0,0,1,0,0,2,0,10,0,0,0,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"(37.59144746148929, 126.88152274735107)",0.00,0.000000,0.000000,[155],82.0,0.0,0.0,37.591447,126.881523,21,1,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,1,0,6,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"(37.59146778178655, 126.88265470326604)",0.00,0.000000,0.000000,[155],36.0,0.0,0.0,37.591468,126.882655,21,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,"(37.70445091519225, 126.75467703178417)",169.25,127.277778,296.527778,[346],3562.0,5665.0,0.0,37.704451,126.754677,28,0,0,0,3,0,0,1,0,12,1,0,8,0,0,2,0,0,0,0,2,0,0,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3230,"(37.704472506609626, 126.75581060819373)",169.25,127.277778,296.527778,[346],2661.0,3486.0,0.0,37.704473,126.755811,27,0,0,0,0,0,0,1,0,12,1,0,9,0,0,9,0,0,0,0,3,0,0,0,2,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3231,"(37.70466633943072, 126.7660128502553)",118.25,101.500000,219.750000,[351],7713.0,21185.0,0.0,37.704666,126.766013,31,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,6,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3232,"(37.70468782197291, 126.76714643873707)",118.25,101.500000,219.750000,[351],6526.0,21185.0,0.0,37.704688,126.767146,33,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,4,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
lscp_new.to_pickle('/content/drive/My Drive/Compas/compas/csv 파일 여기로!/Month_bikepoint_lscp_all_300_도로명.pickle')

In [ ]:
lscp_new.to_pickle('/content/drive/My Drive/Compas/compas/이소연 자료/Month_bikepoint_lscp_all_300_도로명.pickle')

### 2. 합

In [ ]:
lscp_new.to_pickle('/content/drive/My Drive/Compas/compas/csv 파일 여기로!/Month_Sum_bikepoint_lscp_all_300_도로명.pickle')

In [ ]:
lscp_new.to_pickle('/content/drive/My Drive/Compas/compas/이소연 자료/Month_Sum_bikepoint_lscp_all_300_도로명.pickle')